In [13]:
import pandas as pd #DataFrame 용
import numpy as np
import joblib   #모델 불러오기용
import os
import json

from programFiles import keywordManage
from programFiles import Preprocess as pre
from programFiles import Learning

In [3]:
path = "D:\\00.Storage\\Study\\Programing\\Project\\AI_Recommend-Category"+ "\\" + 'ConstData.json'

with open(path, encoding='utf-8') as f:
    json_obj = json.load(f)

json_obj = json_obj["Run"]

del path
del f

In [4]:
fileName = json_obj["runFileDirectory"] + "\\" + json_obj["runFileName"] #파일명
dfExcel = pd.read_excel(fileName)
df = pd.DataFrame(dfExcel, columns= [json_obj["runUseColumn"]]) #사용하는 Column만 추출1

del dfExcel
del fileName

In [5]:
dfInput = keywordManage.loadKeywordList()

for row in df.loc[:, json_obj["runUseColumn"]]:    #상품명 Column 데이터만 추출[aa bb cc, dd,ee]
    keywords = []
    keywordManage.keywordExtract(insList=keywords, strValue=row)
    dfInput = pre.createInputDf(dfInput, keywords)

del keywords
del row

In [6]:
#최근 학습한모델 가져오기
model = joblib.load(Learning.getRecentModel())

#모델에 적용시긴 결과 가져오기

result = pd.DataFrame(model.predict(dfInput))
result['origin'] = df

In [14]:
resultData = model.predict_proba(dfInput)
rateList = []

for row in resultData:
    maxValue = 0
    for value in row:
        if (maxValue < value):
            maxValue = value
    rateList.append(maxValue)

rateList = format(np.argmax(model.predict_proba(dfInput), axis=1))

result['Rate'] = rateList

result.loc[result['Rate'] < 0.6, 0] = ''